In [3]:
data_label = 'capsule'

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import sys
sys.argv = [sys.argv[0], '--mixing', '--sched', 'mvtec_single_out/' +data_label]# '--loss', 'r1'

import argparse
import random
import math

from tqdm import tqdm
from PIL import Image

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable, grad
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils

from dataset import MultiResolutionDataset2
from model import StyledGenerator, Discriminator, Encoder, ShortDiscriminator, StyledGenerator2

from sklearn.manifold import TSNE
from sklearn.datasets import load_digits
from scipy.spatial.distance import pdist
from sklearn.manifold.t_sne import _joint_probabilities
from scipy import linalg
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import squareform
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import seaborn as sns
import imgaug as ia
import imgaug.augmenters as iaa

import numpy as np
import matplotlib.pyplot as plt
import cv2 
from scipy.spatial import distance

def requires_grad(model, flag=True):
    for p in model.parameters():
        p.requires_grad = flag


def accumulate(model1, model2, decay=0.999):
    par1 = dict(model1.named_parameters())
    par2 = dict(model2.named_parameters())

    for k in par1.keys():
        par1[k].data.mul_(decay).add_(1 - decay, par2[k].data)


def sample_data(dataset, batch_size, image_size=4):
    dataset.resolution = image_size
    loader = DataLoader(dataset, shuffle=True, batch_size=batch_size, num_workers=1, drop_last=True)

    return loader


def adjust_lr(optimizer, lr):
    for group in optimizer.param_groups:
        mult = group.get('mult', 1)
        group['lr'] = lr * mult
        
from PIL import Image
image = Image.open('data/test/capsule/squeeze/001.png')

resize_image = image.resize((512, 512))
AAA = None
BBB = None
    
    
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance


def sortpts_clockwise(A):
    sortedAc2 = A[np.argsort(A[:,1]),:]
    top2 = sortedAc2[0:2,:]
    bottom2 = sortedAc2[2:,:]
    sortedtop2c1 = top2[np.argsort(top2[:,0]),:]
    top_left = sortedtop2c1[0,:]
    sqdists = distance.cdist(top_left[None], bottom2, 'sqeuclidean')
    rest2 = bottom2[np.argsort(np.max(sqdists,0))[::-1],:]
    return np.concatenate((sortedtop2c1,rest2),axis =0)
        
    
class ImgAugTransform:
    def __init__(self):
        #self.aug = iaa.Sequential([
             #iaa.CoarseDropout(0.1, size_percent=0.5),
             #iaa.CoarseDropout((0.02, 0.1), size_percent=(0.02, 0.05), per_channel=False),
             #iaa.Salt(0.05, size_percent=(0.01, 0.1)) #AndPepper CoarseSalt
            #iaa.BlendAlphaSimplexNoise(iaa.EdgeDetect(1.0))
            #iaa.Cutout(fill_mode="gaussian", fill_per_channel=True)
            #iaa.Salt(0.05)
        #])
        self.aug = iaa.Sequential([
            #iaa.CoarseDropout(0.1, size_percent=0.5),
            #iaa.CoarseDropout((0.02, 0.1), size_percent=(0.02, 0.05), per_channel=False),
            #iaa.Salt(0.05)#(0.05, size_percent=(0.01, 0.1)) #AndPepper CoarseSalt
            iaa.ChangeColorTemperature((1100, 10000)),
            iaa.KeepSizeByResize( iaa.Crop(percent=(0.0, 0.5), keep_size=False) ),
            iaa.Flipud(0.5),
            iaa.Fliplr(0.5),
        ])
        self.mask = np.zeros((512, 512, 1), dtype=np.int8)
    
    def distortion(self, img):
        A = img.shape[0] / 3.0
        w = 2.0 / img.shape[1]

        shift = lambda x: A * np.sin(2.0*np.pi*x * w)

        #for i in range(img.shape[0]):
        #ret = np.roll(np.array(img[:,:,:]), int(shift(i)))
        
        if (np.random.randint(2) == 0):
            for i in range(img.shape[0]):
                img[i,:,0] = torch.tensor( np.roll(img[i,:,0], int(shift(i))) )
                img[i,:,1] = torch.tensor( np.roll(img[i,:,1], int(shift(i))) )
                img[i,:,2] = torch.tensor( np.roll(img[i,:,2], int(shift(i))) )
        else:
            for i in range(img.shape[1]):
                img[:,i,0] = torch.tensor( np.roll(img[:,i,0], int(shift(i))) )
                img[:,i,1] = torch.tensor( np.roll(img[:,i,1], int(shift(i))) )
                img[:,i,2] = torch.tensor( np.roll(img[:,i,2], int(shift(i))) )
            
        return img

    def __call__(self, img):
        img = np.array(img)
        tile_size = img.shape[0]
        step = int(np.log2(tile_size / 4.0))
        
        n = int(np.random.rand(1) * 7 + 3)
        xy = np.random.uniform(low=0.2, high=0.8, size=(2))
        wh = np.random.uniform(low=0.2, high=0.8, size=(2))

        self.mask[:] = 0
        polys = np.random.rand(10*4).reshape(10, 4)
        wh = np.random.uniform(low=0.2, high=0.8, size=(2))
        n = 5
        tile_size = 512
        max_n = np.random.randint(10)
        #num = np.random.randint(5)
        for i in range(max_n+1):
            cv2.line(self.mask,(int(polys[i, 0]*tile_size), int(polys[i, 1]*tile_size)),
                        (int(polys[i, 2]*tile_size), int(polys[i, 3]*tile_size)),(1),1*1)
        polys = np.random.rand(10*4).reshape(10, 4)
        wh = np.random.uniform(low=0.2, high=0.8, size=(2))
        for i in range(max_n+1):
            cv2.line(self.mask,(int(polys[i, 0]*tile_size), int(polys[i, 1]*tile_size)),
                        (int(polys[i, 2]*tile_size), int(polys[i, 3]*tile_size)),(1),2*2)
        polys = np.random.rand(10*4).reshape(10, 4)
        wh = np.random.uniform(low=0.2, high=0.8, size=(2))
        for i in range(max_n+1):
            cv2.line(self.mask,(int(polys[i, 0]*tile_size), int(polys[i, 1]*tile_size)),
                        (int(polys[i, 2]*tile_size), int(polys[i, 3]*tile_size)),(1),3*3)
        
        #self.mask = 1-self.mask
        
        ret_mask = cv2.resize(self.mask.astype(np.uint8), dsize=(img.shape[0], img.shape[1]), interpolation=cv2.INTER_CUBIC)
        ret_mask = ret_mask[:, :, None]
        img_anom = img
        #if (np.random.randint(2) == 0):
        #    img_anom = (self.distortion( np.array(img_anom).astype(np.uint8) ).copy()).astype(np.float32)
        #else:
        #    pass
        img_anom = (self.aug.augment_image( np.array(img_anom).astype(np.uint8) ).copy()).astype(np.float32)
        #self.mask[:] = 1
        ret = (img - ret_mask*img) + (ret_mask * img_anom)#img_anom#(img - self.mask*img) + (self.mask * img_anom)
        
        return np.concatenate((ret, ret_mask), axis=2)

ia_transform = ImgAugTransform()

code_size = 512
base_batch_size = 2 #16
n_critic = 1

parser = argparse.ArgumentParser(description='Progressive Growing of GANs')

parser.add_argument('path', type=str, help='path of specified dataset')
parser.add_argument(
    '--phase',
    type=int,
    default=20_000, #600_000
    help='number of samples used for each training phases',
)
parser.add_argument('--lr', default=0.001, type=float, help='learning rate')
parser.add_argument('--sched', action='store_true', help='use lr scheduling')
parser.add_argument('--init_size', default=512, type=int, help='initial image size') #2^9
parser.add_argument('--max_size', default=512, type=int, help='max image size')
parser.add_argument(
    '--ckpt', default='checkpoint/'+data_label+'/train_step-7.model', type=str, help='load from previous checkpoints'
)
parser.add_argument(
    '--no_from_rgb_activate',
    action='store_true',
    help='use activate in from_rgb (original implementation)',
)
parser.add_argument(
    '--mixing', action='store_true', help='use mixing regularization'
)
parser.add_argument(
    '--loss',
    type=str,
    default='wgan-gp',
    choices=['wgan-gp', 'r1'],
    help='class of gan loss',
)

args = parser.parse_args()

#encoder = nn.DataParallel(
#    Encoder(from_rgb_activate=not args.no_from_rgb_activate)
#).cuda()
#generator = nn.DataParallel(StyledGenerator(code_size)).cuda()

g2_running = StyledGenerator2(code_size).cuda()

g_running = StyledGenerator(code_size).cuda()
#g_running.train(False)

e_running = Encoder(from_rgb_activate=not args.no_from_rgb_activate).cuda()
#e_running.train(False)

g2_optimizer = optim.Adam(
    g2_running.generator.parameters(), lr=args.lr, betas=(0.0, 0.99)
)
g2_optimizer.add_param_group(
    {
        'params': g2_running.style.parameters(),
        'lr': args.lr * 0.01,
        'mult': 0.01,
    }
)

g_optimizer = optim.Adam(
    g_running.generator.parameters(), lr=args.lr, betas=(0.0, 0.99)
)
g_optimizer.add_param_group(
    {
        'params': g_running.style.parameters(),
        'lr': args.lr * 0.01,
        'mult': 0.01,
    }
)

e_optimizer = optim.Adam(e_running.parameters(), lr=args.lr, betas=(0.0, 0.99))

#accumulate(g_running, generator.module, 0)
#accumulate(e_running, encoder.module, 0)

assert args.ckpt is not None
ckpt = torch.load(args.ckpt)

#generator.module.load_state_dict(ckpt['generator'])
#g_running.load_state_dict(ckpt['g_running'])
#g_optimizer.load_state_dict(ckpt['g_optimizer'])


transform = transforms.Compose(
    [
        #transforms.RandomHorizontalFlip(),
        ia_transform,
        #transforms.ToTensor(),
        #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True),
    ]
)

transform2 = transforms.Compose(
    [
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True),
    ]
)

dataset = MultiResolutionDataset2(args.path, transform, transform2)

if args.sched:
    args.lr = {128: 0.0015, 256: 0.002, 512: 0.003, 1024: 0.003}
    args.batch = {4: 128, 8: 64, 16: 32, 32: 16, 64: 8, 128: 8, 256: 2, 512: 2} #8

else:
    args.lr = {}
    args.batch = {}

args.gen_sample = {512: (8, 4), 1024: (4, 2)}

args.batch_default = base_batch_size * 2


def toImage(tensor):
    A = tensor.clone()
    A -= A.min()
    A /= A.max()
    
    return A


In [6]:
ckpt = torch.load('checkpoint/'+data_label+'/train_step-7.model')
g_running.load_state_dict(ckpt['g_running'])

<All keys matched successfully>

In [7]:
import numpy as np

np.random.seed(None)
st0 = np.random.get_state()
# draw some random numbers
print(np.random.randint(0, 100, 10))
np.random.set_state(st0)
print(np.random.randint(0, 100, 10))


[36  1 33 90  6 55 38 23 16 93]
[36  1 33 90  6 55 38 23 16 93]


In [8]:
import cv2 as cv
import os
from PIL import Image
import matplotlib.pyplot as plt

def get_holes(image, thresh):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    im_bw = cv.threshold(gray, thresh, 255, cv.THRESH_BINARY)[1]
    im_bw_inv = cv.bitwise_not(im_bw)

    contour, _ = cv.findContours(im_bw_inv, cv.RETR_CCOMP, cv.CHAIN_APPROX_SIMPLE)
    for cnt in contour:
        cv.drawContours(im_bw_inv, [cnt], 0, 255, -1)

    nt = cv.bitwise_not(im_bw)
    im_bw_inv = cv.bitwise_or(im_bw_inv, nt)
    return im_bw_inv


def remove_background(image, thresh, scale_factor=.25, kernel_range=range(1, 15), border=None):
    border = border or kernel_range[-1]

    holes = get_holes(image, thresh)
    small = cv.resize(holes, None, fx=scale_factor, fy=scale_factor)
    bordered = cv.copyMakeBorder(small, border, border, border, border, cv.BORDER_CONSTANT)

    for i in kernel_range:
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (2*i+1, 2*i+1))
        bordered = cv.morphologyEx(bordered, cv.MORPH_CLOSE, kernel)

    unbordered = bordered[border: -border, border: -border]
    mask = cv.resize(unbordered, (image.shape[1], image.shape[0]))
    fg = cv.bitwise_and(image, image, mask=mask)
    return fg, mask

def get_score(generator2, encoder, generator):
    num_file = 0
    r_list = []
    for root, dirs, files in os.walk("./test/"+data_label+"/test/"):
        path = root.split(os.sep)
        #print((len(path) - 1) * '---', os.path.basename(root))
        if path[-1] == 'good' or path[-1] == '.ipynb_checkpoints':
            continue
        r_list_sub = []
        for file in (sorted(files[:])):
            if '.png' == file[-4:] and path[-1] != '':
                #print()
                # Open the image form working directory
                anomaly = Image.open('./test/'+data_label+'/test/' + str(path[-1]) + '/' + file).convert('RGB')
                gt = Image.open('./test/'+data_label+'/gt/' + str(path[-1]) + '/' + file[:-4] + '_mask.png')
                gt = gt.resize((512, 512))
                anomaly = anomaly.resize((512, 512))

                gen_in1, gen_in2 = torch.randn(2, 1, code_size, device='cuda').chunk(
                        2, 0
                    )
                gen_in1 = gen_in1.squeeze(0)

                ground_truth = 1 - (torch.from_numpy(np.array(gt)[:, :]).permute(0, 1) / 255.0)
                anomaly_image = torch.from_numpy(np.array(anomaly)[None, :, :, :]).permute(0, 3, 1, 2) / 255.0
                anomaly_image = anomaly_image.cuda()

                anomaly_image = anomaly_image
                anomaly_image = anomaly_image * 2
                anomaly_image = anomaly_image - 1.0

                #out = encoder(anomaly_image, step=step, alpha=alpha)
                (out, skiplist) = encoder(anomaly_image, step=step, alpha=alpha)
                skiplist = skiplist[1:]
                skiplist.insert(0, None)

                out_orin, fake_image     = generator([gen_in1], styles=[out[:, :512]], step=step, alpha=alpha)
                out_orin, (fake_image2, mask)     = generator2(skiplist, fake_image, anomaly_image, [gen_in1], styles=[out[:, 512:]], step=step, alpha=alpha)

                gt_image = torch.from_numpy(np.array(gt)[None, :, :]) / 255.0
                gt_image_im =torch.from_numpy(np.array(( toImage(torch.sum( gt_image.data[:, :, :], axis = 0) ) ) ))


                fake_image_norm = (1 - mask) * fake_image + mask * anomaly_image


                anomaly_image_norm = anomaly_image
                anomaly_image_norm = (anomaly_image_norm - torch.min(anomaly_image_norm))
                anomaly_image_norm = anomaly_image_norm / (torch.max(anomaly_image_norm) + 1e-6)

                img_diff = mask
                img_diff = torch.sum( torch.abs(img_diff), axis = 0)
                img_diff = torch.from_numpy(np.array(( toImage(torch.sum( img_diff.data.cpu()[:, :, :], axis = 0) ) ) )).permute(0, 1)

                fg = torch.from_numpy(np.array(( toImage( anomaly_image.data.cpu()[0, :, :, :] ) ) )).permute(1, 2, 0)
                img = np.array(fg*255).astype(np.uint8)
                nb_img, mask_fg = remove_background(img, 120)

                kernel = np.ones((9,9),np.uint8)
                img_diff = cv2.dilate(np.array(img_diff),kernel,iterations = 1)
                ret = ((torch.tensor(img_diff) > 0.2) & (mask_fg > 0) )
                gt = (gt_image_im > 0)

                score = float(torch.sum ( ret & gt ).item()) / torch.sum ( ret | gt ).item()
                
                r_list_sub.append(score)
            
        r_list.append(r_list_sub)
        
    length = 0
    sum_score = 0
    for s_list in r_list:
        length += len(s_list)
        sum_score += sum(s_list)
    
    return sum_score / length

In [10]:
generator2 = g2_running
encoder = e_running
generator = g_running

losses = []
scores = []
scores.append(0)
step = int(math.log2(args.init_size)) - 2
resolution = 4 * 2 ** step


resize_image = image.resize((512, 512))
anomaly_image = torch.from_numpy(np.array(resize_image)[None, :, :, :]).permute(0, 3, 1, 2) / 255.0
anomaly_image = anomaly_image.cuda()
anomaly_image = anomaly_image
anomaly_image = anomaly_image * 2
anomaly_image = anomaly_image - 1.0

loader = sample_data(
    dataset, args.batch.get(resolution, args.batch_default), resolution
)
data_loader = iter(loader)

adjust_lr(g_optimizer, args.lr.get(resolution, 0.001))

pbar = tqdm(range(3_000_000)) #3_000_000

requires_grad(generator, False)

gen_loss_val = 0
grad_loss_val = 0

alpha = 1
used_sample = 0

max_step = int(math.log2(args.max_size)) - 2
final_progress = False


for i in pbar:
    if used_sample > args.phase * 2:
        used_sample = 0
        step += 1

        if step > max_step:
            step = max_step
            final_progress = True
            ckpt_step = step + 1

        else:
            ckpt_step = step

        resolution = 4 * 2 ** step

        loader = sample_data(
            dataset, args.batch.get(resolution, args.batch_default), resolution
        )
        data_loader = iter(loader)

        torch.save(
            {
                'g_optimizer': g_optimizer.state_dict(),
                'g_running': g_running.state_dict(),
                'e_running': e_running.state_dict(),
            },
            ('checkpoint/'+data_label+'/encdec/train_step-' + str(ckpt_step) + '.model'),
        )

        adjust_lr(g_optimizer, args.lr.get(resolution, 0.001))
        #gc.collect()
    try:
        real_image, anomaly_img, mask_gt = next(data_loader)

    except (OSError, StopIteration):
        data_loader = iter(loader)
        real_image, anomaly_img, mask_gt = next(data_loader)

    used_sample += real_image.shape[0]

    b_size = real_image.size(0)
    real_image = real_image.cuda()
    anomaly_img = anomaly_img.cuda()
    mask_gt = mask_gt.unsqueeze(1)
    mask_gt = mask_gt.cuda()
    
   

    # Encoder 
    encoder.zero_grad()

    requires_grad(encoder, True)
    requires_grad(generator2, True)
    requires_grad(generator, False)

    gen_in1, gen_in2 = torch.randn(2, b_size, code_size, device='cuda').chunk(
            2, 0
        )
    gen_in1 = gen_in1.squeeze(0)
    
    
    anomaly_img = anomaly_img
    anomaly_img = anomaly_img / 255.0
    anomaly_img = anomaly_img - 1.0
    
    (out, skiplist) = encoder(anomaly_img, step=step, alpha=alpha)
    skiplist = skiplist[1:]
    skiplist.insert(0, None)

    out_orin, fake_image     = generator([gen_in1], styles=[out[:, :512]], step=step, alpha=alpha) #styles=[out],
    out_orin2, (fake_image2, mask) = generator2(skiplist, fake_image, anomaly_img, [gen_in1], styles=[out[:, 512:]], step=step, alpha=alpha)

    fake_image_norm = fake_image2
    anomaly_image_norm = real_image

    loss = nn.MSELoss(reduction = 'mean')
    diff_norm = loss(fake_image_norm, anomaly_image_norm)
    diff_norm2 = loss(fake_image, anomaly_image_norm)
    diff_norm3 = loss(mask, mask_gt)
    diff_norm4 = loss(fake_image, fake_image_norm)
    recon_loss_pixel = torch.mean(diff_norm) + torch.mean(diff_norm2) + torch.mean(diff_norm3)+ torch.mean(diff_norm4)
    
    loss_norm = torch.norm(mask)
    
    recon_loss = recon_loss_pixel 

    if i%10 == 0:
        encoder_loss_val = recon_loss.item()
        losses.append(encoder_loss_val)
    
    recon_loss.backward()
    e_optimizer.step()
    
    requires_grad(generator2, False)
    requires_grad(encoder, False)
    requires_grad(generator, False)
    
    # Encoder End

    if (i + 1) % 1000 == 0:
        images = []

        gen_i, gen_j = args.gen_sample.get(resolution, (10, 5))
        
        
        anom = anomaly_img 
        (out_tmp, list_tmp) = encoder(anom, step=step, alpha=alpha)

        images.append(anomaly_img.data.detach().cpu())
        images.append((anom*2-1).data[:1].detach().cpu())
        
        list_tmp = list_tmp[1:]
        list_tmp.insert(0, None)

        fk2 = g2_running(
            list_tmp, fake_image[:], anom, [gen_in1], styles=[out[:, 512:]], step=step, alpha=alpha
        )
        fk2_mask = torch.cat( [fk2[1][1].data[:1].detach().cpu(), fk2[1][1].data[:1].detach().cpu(),
                               fk2[1][1].data[:1].detach().cpu()], axis=1)

        fk3_mask = torch.cat( [fk2[1][1].data[1:].detach().cpu(), fk2[1][1].data[1:].detach().cpu(),
                               fk2[1][1].data[1:].detach().cpu()], axis=1)      

        with torch.no_grad():
            for _ in range(1):
                images.append(
                    g_running(
                        [out[:, :512]], styles=[out[:, :512]], step=step, alpha=alpha
                    )[1].data.detach().cpu() 
                )
                images.append(
                    g_running(
                        [out_tmp[:, :512]], styles=[out_tmp[:, :512]], step=step, alpha=alpha
                    )[1].data[:1].detach().cpu() 
                )
                images.append(
                    fk2[1][0].data[:1].detach().cpu()
                )
                images.append(
                    fk2_mask
                )
                
                images.append(
                    fk2[1][0].data[1:].detach().cpu()
                )
                images.append(
                    fk3_mask
                )
        utils.save_image(
            torch.cat(images, 0),
            'sample/'+data_label+'/encdec/'+str(i + 1).zfill(6)+ '.png',
            nrow=gen_i,
            normalize=True,
            range=(-1, 1),
        )
        
        latest_score = get_score(generator2, encoder, generator)
        
        if latest_score > max(scores):
            print(latest_score)
            print(str(i + 1).zfill(6))
            torch.save(
                g2_running.state_dict(), 'checkpoint/'+data_label+'/encdec/bestg2.model'
            )
            torch.save(
                e_running.state_dict(), 'checkpoint/'+data_label+'/encdec/beste.model'
            )
        scores.append(latest_score)

    if (i + 1) % 20000 == 0:
        torch.save(
            g2_running.state_dict(), 'checkpoint/'+data_label+'/enc'+str(i + 1).zfill(6)+'g2.model'
        )
        torch.save(
            e_running.state_dict(), 'checkpoint/'+data_label+'/enc'+str(i + 1).zfill(6)+'e.model'
        )
        
    state_msg = (
        f'Size: {4 * 2 ** step}; E: {encoder_loss_val:.3f}; G: {gen_loss_val:.3f}; '
        f' Grad: {grad_loss_val:.3f}; Alpha: {alpha:.5f}'
    )

    pbar.set_description(state_msg)

Size: 512; E: 1.329; G: 0.000;  Grad: 0.000; Alpha: 1.00000:   0%|          | 4/3000000 [00:01<393:28:30,  2.12it/s]


KeyboardInterrupt: 

In [9]:
st0

('MT19937',
 array([2147483648, 1973952246, 2700483622, 3062020305, 2909344801,
        1116939581, 1008371610, 1249036225,  861768517, 2505248493,
        3593500672,  227497116, 2298433495, 3516138394, 1044592035,
        2361457843, 1291448833, 3685614512,  953746249, 3087288407,
        3122638999,  786488974, 4197158251, 3744250015, 3836680439,
        3384097227, 1656112966, 1328243768, 3803903442, 1794590777,
        3145332185, 1201569261,  535702486, 3007142371, 1021677493,
        4052770243, 3079164736, 2840641662, 3374635812, 4017673419,
        2440034551, 1342282559, 2906900979,  780450199, 1347469560,
        3690854515, 2138950941, 2727648487, 2107678168, 2212443201,
        3491437067, 3083886467,  198888914, 3706326276, 1036374021,
         558837500, 1342537693, 4139764033, 2584649342, 2992058692,
         840155387, 2488245761, 1946839338, 2429375166, 2799784793,
        4165585943, 2609904223, 2235539820,   78207713, 2314463581,
         512636430, 2253907428, 2314

In [10]:
scores

[0,
 0.07681572528798353,
 0.15823801174198843,
 0.14375657892854732,
 0.1649379840151192,
 0.24000147749409725,
 0.2674371448776942,
 0.25342091404310474,
 0.3026808756594139,
 0.29365830074689714,
 0.19971747094819498,
 0.19560848506239964,
 0.31294718106836894,
 0.3727348359223772,
 0.363653746723367,
 0.3805039425279266,
 0.4316559131551962,
 0.4271487367949082,
 0.4398161569591399,
 0.3041939228740437,
 0.41823271521052596,
 0.4057650560917065,
 0.3963421301513099,
 0.29153030103173155,
 0.3837023667021972,
 0.3695862069284389,
 0.4240676969617158,
 0.3718329493472246,
 0.3989592922881014,
 0.41507439142530356,
 0.38584033947351576,
 0.36049771749190007,
 0.42236058212034566,
 0.441476272858257,
 0.421281103107415,
 0.44011783343213096,
 0.41338076156588804,
 0.4175040939338306,
 0.38260983803332516,
 0.3586686352806997,
 0.3692592179127893,
 0.41976986628526897,
 0.4051606680001207,
 0.38355359515252024,
 0.3810865750730608,
 0.389684930817785,
 0.37531505921337766,
 0.4506503942